In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
# TOKEN GENERATION
CLIENT_ID = "c87105581fca41ec838b91d6fffc2096"
CLIENT_SECRET = "41a9e1f80f334bfbb3197e89a411485a"
AUTH_URL = "https://accounts.spotify.com/api/token"

# POST
auth_response = requests.post(AUTH_URL, {
    "grant_type" : "client_credentials",
    "client_id" : CLIENT_ID,
    "client_secret" : CLIENT_SECRET,
})

#CONVERT RESPONSE TO JSON
auth_response_data = auth_response.json()

#SAVE TOKEN
token = auth_response_data["access_token"]

In [10]:
#LOOP FOR PULLING PLAYLIST METADATA
offset = 0
limit = 100
MAX = 870
is_df_ready = False
while True:

    if offset > MAX:
        offset = MAX
    response = requests.get("https://api.spotify.com/v1/playlists/6qBlv0CvTkdwU1iBuMImzs/tracks?limit={}&offset={}".format(limit,offset), headers = {"Accept":"application/json", "Content-Type":"application/json","Authorization":"Bearer {}".format(token)})
    if is_df_ready:
        this_response = pd.DataFrame(response.json()["items"])
        all_songs = pd.concat([all_songs,this_response])
    else:
        all_songs=pd.DataFrame(response.json()["items"])
        is_df_ready = True
    offset = offset + limit
    if offset >= MAX:
        break

In [21]:
#EXTRACTING IDS
liked =all_songs["track"]

In [14]:
def extract_id(row):
    return row["uri"].split(":")[2]

In [20]:
track_df = all_songs["track"].apply(extract_id)
track_df.to_csv("LIKED_IDS.csv")